In [25]:
import pandas as pd
import csv
import os
import math

# Parsing Census Data
The goal of this file is to parse data from census tables into an intermediate format so we can merge them into a master table later. 

## Steps:
1. Duplicate the folder (i.e. `S2301`) within `census-data`, and rename the folder to the name of the Census table you are processing
2. Empty the `raw` folder
3. Download census data and place into `raw`. Each year of data should be in its own csv file. Rename each file into `<year>.csv`
4. Update the transformation in the code block below to obtain the desired columns, and rename them if necessary
    - For most tables, you will only need to lookup the column code of the feature you are looking to obtain, and match that to a readable name in `features`
5. Verify the transformation is correct using the `df.head()` statement already placed there for you. If it looks good, the data is good!

In [26]:
def process_file(filename: str):
    year = filename[:filename.find(".")]
    df = pd.read_csv('./raw/' + filename)
    df['cfips'] = df['GEO_ID'].apply(lambda x: x[-5:])

    # Map column codes to human-readable names. 
    # ==============
    features = {
        'B25085_001E': 'total', 
        'B25085_027E': 'gt2.0', 
        'B25085_026E': '1.5-2.0', 
        'B25085_025E': '1.0-1.5',
        'B25085_024E': '0.75-1.0',
        'B25085_023E': '0.5-0.75',
        'B25085_022E': '0.4-0.5',
        'B25085_021E': '0.3-0.4',
        'B25085_020E': '0.25-0.3',
        'B25085_019E': '0.2-0.25',
        'B25085_018E': '0.175-0.2',
        'B25085_017E': '0.15-0.175',
        'B25085_016E': '0.125-0.15',
        'B25085_015E': '0.1-0.125',
    }

    df = df.iloc[1:]
    df = df.filter(items=['cfips'] + list(features.keys()))
    df = df.rename(columns=features)
    for feature in features.values():
        df[feature] = df[feature].apply(lambda x: int(x))
    df['pct_house_price_gt1mill'] = (df['gt2.0'] + df['1.5-2.0'] + df['1.0-1.5'])/df['total'] * 100
    df['pct_house_price_500k_1mill'] = (df['0.5-0.75'] + df['0.75-1.0'])/df['total'] * 100
    df['pct_house_price_250k_500k'] = (df['0.25-0.3'] + df['0.3-0.4'] + df['0.4-0.5'])/df['total'] * 100
    df['pct_house_price_100k_250k'] = (df['0.1-0.125'] + df['0.125-0.15'] + df['0.15-0.175'] + df['0.175-0.2'] + df['0.2-0.25'])/df['total'] * 100
    df = df.filter(items=['cfips', 'pct_house_price_gt1mill', 'pct_house_price_500k_1mill', 'pct_house_price_250k_500k', 'pct_house_price_100k_250k'])
    for feature in ['pct_house_price_gt1mill', 'pct_house_price_500k_1mill', 'pct_house_price_250k_500k', 'pct_house_price_100k_250k']:
        df[feature] = df[feature].apply(lambda x: 0 if math.isnan(x) else x)
    # ==============

    print(df.head())
    df.to_csv('./parsed/' + year + '.csv', index=False)

for filename in os.listdir("./raw"):
    process_file(filename)

   cfips  pct_house_price_gt1mill  pct_house_price_500k_1mill  \
1  01001                 0.000000                    0.000000   
2  01003                 3.076923                    7.065527   
3  01005                 2.631579                    0.000000   
4  01007                 0.000000                   19.801980   
5  01009                 2.127660                    0.000000   

   pct_house_price_250k_500k  pct_house_price_100k_250k  
1                  14.761905                  44.761905  
2                  32.877493                  42.962963  
3                  18.859649                  41.666667  
4                   0.000000                  44.554455  
5                  14.893617                  20.744681  
   cfips  pct_house_price_gt1mill  pct_house_price_500k_1mill  \
1  01001                 0.000000                    0.000000   
2  01003                 1.095008                    6.892110   
3  01005                 2.390438                    0.000000   
4